#### <font color=red>Important: please install folium (pip install folium) and geopy (pip install geopy) to run this workbook successfully</font>


# Python project: Mapping stores and their customers
In this project, we have geographical data on several different stores and their customers in the Greater London area. We build an application that visualizes the geographical location of each store and its respective customers.

We proceed with the following steps:
1. Import and clean data set
2. Find customers of each store and generate geographic location of each store and its customers
3. Create map

### Step 1: Import and clean data set

In [1]:
# Import data set from S3 bucket
import pandas as pd
df = pd.read_excel('https://s3.amazonaws.com/dmfaproject4flightsdata/LaptopSalesLL.xlsx', sheet_name='Sales Data With Lat&Long')
df = df.dropna()
df = df.reset_index(drop = True)

In [2]:
# Manipulate data set - prepare to find unique stores and customers
df2 = df[['Customer Longitude', 'Customer Latitude', 'Store Latitude', 'Store Longitude']].copy()
df2['CustomerID'] = df2["Customer Longitude"].map(str) + df2["Customer Latitude"].map(str)
df2['StoreID'] = df2["Store Longitude"].map(str) + df2["Store Latitude"].map(str)
df2.head()

,Customer Longitude,Customer Latitude,Store Latitude,Store Longitude,CustomerID,StoreID
0,-0.098731,51.512579,51.500186,-0.070057,-0.09873151.512579,-0.0700570000000000151.500186
1,-0.140912,51.464129,51.442082,-0.149046,-0.14091251.464129,-0.14904651.442082
2,-0.083435,51.512699,51.529346,-0.045410,-0.08343551.512699,-0.0454151.529346000000004
3,-0.129897,51.500787,51.500186,-0.070057,-0.12989751.500787,-0.0700570000000000151.500186
4,-0.103737,51.51212,51.490255,-0.144474,-0.10373751.51212,-0.14447451.490255


In [3]:
# Adjust data types
df2['Customer Longitude'] = pd.to_numeric(df2['Customer Longitude'], errors='coerce')
df2['Customer Latitude'] = pd.to_numeric(df2['Customer Latitude'], errors='coerce')
df2.dtypes
df2 = df2.dropna()
df2 = df2.reset_index(drop = True)

In [4]:
# Count unique stores and customers
a = df2['StoreID'].unique()
b = df2['CustomerID'].unique()

In [5]:
# Create labels for unique stores and customers
for i in range(len(a)):
    df2.loc[df2.StoreID == a[i], 'StoreName'] = i
    
for i in range(len(b)):
    df2.loc[df2.CustomerID == b[i], 'CustomerName'] = i

### Step 2: Find customers of each store and generate geographic location of each store and its customers

In [6]:
# Find out which customers visited which store
for i in [0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0]:
    df2.loc[df2.StoreName == i, 'WasInStoreX'] = i
df2.head()

,Customer Longitude,Customer Latitude,Store Latitude,Store Longitude,CustomerID,StoreID,StoreName,CustomerName,WasInStoreX
0,-0.098731,51.512579,51.500186,-0.070057,-0.09873151.512579,-0.0700570000000000151.500186,0.0,0.0,0.0
1,-0.140912,51.464129,51.442082,-0.149046,-0.14091251.464129,-0.14904651.442082,1.0,1.0,1.0
2,-0.083435,51.512699,51.529346,-0.045410,-0.08343551.512699,-0.0454151.529346000000004,2.0,2.0,2.0
3,-0.129897,51.500787,51.500186,-0.070057,-0.12989751.500787,-0.0700570000000000151.500186,0.0,3.0,0.0
4,-0.103737,51.512120,51.490255,-0.144474,-0.10373751.51212,-0.14447451.490255,3.0,4.0,3.0


In [7]:
# Creating list of coordinates for all stores
allstores_lat = []
allstores_lon = []

# Define dictionaries
dict = {'0':0.0,'1':1.0, '2':2.0, '3':3.0, '4':4.0,'5':5.0, '6':6.0, '7':7.0, '8':8.0,'9':9.0, '10':10.0, '11':11.0, '12':12.0, '13':13.0, '14':14.0}
storeslatdict = {}
storeslondict = {}

# Loop over all stores
for i in ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14']:
    
    
    alllat = df2.loc[df2.StoreName == dict[i], 'Store Latitude']
    alllat = alllat.reset_index(drop=True)
    allstores_lat.append(alllat[0])


    alllon = df2.loc[df2.StoreName == dict[i], 'Store Longitude']
    alllon = alllon.reset_index(drop=True)
    allstores_lon.append(alllon[0])
    

print(allstores_lat)
print(len(allstores_lat))
print(allstores_lon)
print(len(allstores_lon))

[51.500186, 51.442082, 51.529346000000004, 51.490255, 51.483307, 51.461802, 51.398191, 51.551091, 51.485033, 51.500787, 51.541776, 51.600827, 51.51937, 51.418946999999996, 51.593016]
15
[-0.07005700000000001, -0.149046, -0.04541, -0.144474, -0.025775, -0.199785, -0.09363099999999999, -0.137239, -0.279066, -0.129897, 0.037959, -0.194725, -0.211483, -0.305625, -0.070008]
15


In [8]:
# Creating list of unique customers for each store

# Define dictionaries
dict = {'0':0.0,'1':1.0, '2':2.0, '3':3.0, '4':4.0,'5':5.0, '6':6.0, '7':7.0, '8':8.0,'9':9.0, '10':10.0, '11':11.0, '12':12.0, '13':13.0, '14':14.0}
latdict = {}
londict = {}

# Get unique customers for each store by looping over each store
for i in ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14']:
    cust = []
    unique_customers_all = []
    cust = df2.loc[df2.WasInStoreX == dict[i], ['Customer Latitude', 'Customer Longitude', 'CustomerName']]
    unique_customers_all = cust.CustomerName.unique().tolist()
    

# Get lat and lon for unique customers for each store
    x = []
    y = []
    clat_all = []
    clon_all = []
    
    # Loop over each unique customer
    for j in unique_customers_all:
        x = cust.loc[cust['CustomerName'] == j, 'Customer Latitude']
        x = x.reset_index(drop=True)
        clat_all.append(x[0])

        y = cust.loc[cust['CustomerName'] == j, 'Customer Longitude']
        y = y.reset_index(drop=True)
        clon_all.append(y[0])
    
    latdict[i] = clat_all
    londict[i] = clon_all

### Step 3: Create map visualizing location of each store and its customers

In [9]:
# Create map for customers that visited each store
import folium
import geopy.distance

dict = {'0':0.0,'1':1.0, '2':2.0, '3':3.0, '4':4.0,'5':5.0, '6':6.0, '7':7.0, '8':8.0,'9':9.0, '10':10.0, '11':11.0, '12':12.0, '13':13.0, '14':14.0}
dict2 = {'0':0,'1':1, '2':2, '3':3, '4':4,'5':5, '6':6, '7':7, '8':8,'9':9, '10':10, '11':11, '12':12, '13':13, '14':14}

# Iterare through all stores to create a separate map for each store
for i in ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14']:
   
    # Create map object
    m = folium.Map(location=[51.500186,-0.070057], zoom_start=11)
    
    # Create matching lat and lon lists for customers
    clat = latdict[i]
    clon = londict[i]
    
    within2km = []
    distance_summary = []

    # Iterate through  unique customers for each store to create customer markers & lines to respective store
    for j in range(len(clat)):
        
        # Calculate distance between each customer and the store
        distance = geopy.distance.distance([clat[j],clon[j]],[allstores_lat[dict2[i]],allstores_lon[dict2[i]]]).km
        
        if distance < 2.01:
            within2km.append(distance)
        distance_summary.append(distance)
        
        # Define text for tooltip and popup of each customer
        tooltip = 'This is a customer of your selected store. Distance to store: {0:4.2f}km'.format(distance)
        customer_popup = 'Latitude: {}. Longitude {}'.format(clat[j],clon[j])
        
        # Create customer markers and lines from customers to store
        folium.PolyLine(locations=[[clat[j], clon[j]],[allstores_lat[dict2[i]],allstores_lon[dict2[i]]]], color='blue').add_to(m)
        folium.Circle([clat[j], clon[j]], radius = 20, color = 'blue', popup=customer_popup, tooltip = tooltip).add_to(m)
        
        
    # Create store marker for base store
    # Define text for tooltip and popup
    count_within2km = len(within2km)
    max_distance = max(distance_summary)
    basestore_popup = 'Customers within 2 km radius: {0}. Furthest customer distance: {1:4.2f} km'.format(count_within2km, max_distance)
    
    tooltip = 'This is your selected store. Click for more info'
    
    # Create marker and circle
    folium.Marker([allstores_lat[dict2[i]],allstores_lon[dict2[i]]], icon = folium.Icon(color='green', prefix='fa', icon = 'fa-shopping-cart'), popup=basestore_popup, tooltip = tooltip).add_to(m)
    folium.Circle([allstores_lat[dict2[i]],allstores_lon[dict2[i]]], radius = 2000, color='green', tooltip = 'This shows a 2km radius from your selected store').add_to(m)
    
    
    # Create store marker for all other stores
    storeset = set([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]) - set([dict2[i]])
    tooltip = 'This is another store. Click for more info'
    for j in list(storeset):
        storemapname = "<a href=\"" + 'mapstore_try' + str(j) + '.html'  + "\">" + 'See all customers of this store!' + "</a>"
        folium.Marker([allstores_lat[j], allstores_lon[j]], icon = folium.Icon(color='lightred', prefix='fa', icon ='fa-shopping-cart'), popup=storemapname, tooltip = tooltip).add_to(m)


    # Create map name
    mapname = 'mapstore_try' + i + '.html' 

    # Generate map
    m.save(mapname)

#### You can view the map of stores and customers [here](./mapstore_try12.html).